In [57]:
import pandas as pd
import numpy as np
# import scipy.stat as sps
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import ExcelWriter

In [58]:
%%time
df = pd.read_excel('card transactions.xlsx')

Wall time: 9 s


In [59]:
# summary
numrecords = len(df)
stat_df = pd.DataFrame(df.isnull().sum(axis = 0))
stat_df = stat_df.reset_index().rename(columns = {'index':'variable', 0 :"# NaN"})
stat_df["% populated"] = (1 - stat_df["# NaN"]/numrecords)*100
stat_df["# unique"] = ''

for i in range(len(list(df))):
    stat_df["# unique"][i] = df[list(df)[i]].nunique()
    
stat_df

C:\Users\beizh\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,variable,# NaN,% populated,# unique
0,Recnum,0,100.000000,96753
1,Cardnum,0,100.000000,1645
2,Date,0,100.000000,365
3,Merchnum,3375,96.511736,13091
4,Merch description,0,100.000000,13126
5,Merch state,1195,98.764896,227
6,Merch zip,4656,95.187746,4567
7,Transtype,0,100.000000,4
8,Amount,0,100.000000,34909
9,Fraud,0,100.000000,2


In [60]:
# df.head(10)

In [61]:
##Identify any exclusions, bad records

# Remove single large transaction outlier in Amount
df = df[df['Amount']<3000000]

# Remove all but P transaction type
df = df[df['Transtype'] == 'P']

len(df)

96397

In [62]:
# --------------- Fill in NAs ----------

### Merchnum 
# Transform 0 values into NA
df.loc[df['Merchnum'] == '0', 'Merchnum'] = np.NaN

# fill most frequenct merchnum using merch description
dict1 = df.set_index('Merch description')['Merchnum'].to_dict()
df['Merchnum'].fillna(df['Merch description'].map(dict1), inplace = True)

# fill most frequenct merchnum in this zip
dict1 = dict(zip(df['Merch zip'].dropna(), df['Merchnum']))
df['Merchnum'].fillna(df['Merch zip'].map(dict1), inplace = True)

pd.isnull(df['Merchnum']).sum()


1414

In [63]:
### Merch state and zip
numrecords - (df['Merch state'].isna() == df['Merch zip'].isna()).sum()


3810

In [64]:
### Merch zip
# df.groupby(['Merch description','Merch zip']).size()

# fill NA in Merch zip using Merch description
dict1 = df.set_index('Merch description')['Merch zip'].to_dict()
df['Merch zip'].fillna(df['Merch description'].map(dict1), inplace = True)

# fill NA in Merch zip using Merchnum
dict1 = df.set_index('Merchnum')['Merch zip'].to_dict()
df['Merch zip'].fillna(df['Merchnum'].map(dict1), inplace = True)

# fill NA in Merch zip using Cardnum
dict1 = df.set_index('Cardnum')['Merch zip'].to_dict()
df['Merch zip'].fillna(df['Cardnum'].map(dict1), inplace = True)
# It will automatically choose zip with highest frequency

pd.isnull(df['Merch zip']).sum()


543

In [65]:
dic = dict(zip(df['Cardnum'], df['Merch zip']))
dic[5142310525]

60108.0

In [66]:
### Merch state
# df.groupby(['Merch description','Merch state']).size()
# Each Merch description has one matched state

# fill NA in Merch state using Merch description
dict1 = df.set_index('Merch description')['Merch state'].to_dict()
df['Merch state'].fillna(df['Merch description'].map(dict1), inplace = True)

# fill NA in Merch state using Merchnum
dict1 = df.set_index('Merchnum')['Merch state'].to_dict()
df['Merch state'].fillna(df['Merchnum'].map(dict1), inplace = True)

# fill NA in Merch state using Merch zip

dict1 = dict(zip(df['Merch zip'].dropna(), df['Merch state']))
df['Merch state'].fillna(df['Merch zip'].map(dict1), inplace = True)

# fill NA in Merch state using Cardnum
dict1 = df.set_index('Cardnum')['Merch state'].to_dict()
df['Merch state'].fillna(df['Cardnum'].map(dict1), inplace = True)

pd.isnull(df['Merch state']).sum()


122

In [67]:
# Drop remained NA in three variables
df = df.dropna()
len(df)

94631

In [68]:
# df.groupby(['Cardnum','Date'])['Amount'].value_counts()
# df[df['Cardnum'] == 5142190439]

In [74]:
# --------------- Create variables ----------
### Amount expert variables

# Cardnum
# sum the amount at the same day first and then rolling
new_df = df.join(df.groupby(['Cardnum','Date'])['Amount'].sum(), on = ['Cardnum','Date'],rsuffix='_cardnum_day')
# df.set_index('Date').groupby(['Cardnum'])['Amount_cardnum_day'].rolling(window = '10D').mean().drop_duplicates()


# data_range = lambda x: x.max() - x.min()
# ts.groupby(key).transform(data_range)


Cardnum     Date      
5142110081  2010-11-26     636.200000
            2010-12-27     495.900000
5142110402  2010-01-19     506.500000
            2010-01-31      20.810000
            2010-02-03      25.535000
            2010-02-04     180.943333
            2010-02-04     258.647500
5142110434  2010-10-06    1551.640000
5142110749  2010-04-19      62.940000
            2010-06-13     794.850000
5142110909  2010-01-17     351.240000
            2010-01-20     207.820000
            2010-02-09     301.790000
            2010-03-21     128.000000
            2010-03-22     492.935000
            2010-03-22     614.580000
            2010-04-05     435.470000
            2010-04-12     313.560000
            2010-04-12     272.923333
            2010-04-13     407.055000
            2010-05-18     159.600000
            2010-05-18     159.600000
            2010-06-12     288.000000
            2010-07-07    1535.400000
            2010-07-12     793.575000
            2010-12-20     